## Imports

In [1]:
import pandas as pd
import xarray as xr

In [2]:
def pre_csv(df):
    # Wandelt die Spalte 'dtm' in datetime-Objekte um
    df.dtm = pd.to_datetime(df.dtm)
    return df

def pre_dwd(df):
    # Konvertiert ein xarray Dataset in einen DataFrame und setzt den Index zurück
    df = df.to_dataframe().reset_index()
    # Lokalisiert 'ref_datetime' auf UTC
    df.ref_datetime = df.ref_datetime.dt.tz_localize("UTC")
    # Berechnet 'valid_datetime' basierend auf 'ref_datetime' und einer Zeitspanne in Stunden
    df.valid_datetime = df.ref_datetime + df.valid_datetime * pd.Timedelta(1, "h")
    return df

def pre_ncep(df):
    return df


In [3]:
# Einlesen der CSV Datei
df_20200920_20231027 = pre_csv(pd.read_csv('data/Energy_Data_20200920_20231027.csv'))

# Einlesen des Datasets und Umbenennen von Spalten, falls vorhanden
df_dwd_demand = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_demand_20200920_20231027.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_hornsea = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_pes10 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)

df_ncep_demand = pre_ncep(xr.open_dataset("data/ncep_gfs_demand_20200920_20231027.nc"))
df_ncep_hornsea = pre_ncep(xr.open_dataset("data/ncep_gfs_hornsea_1_20200920_20231027.nc"))
df_ncep_pes10 = pre_ncep(xr.open_dataset("data/ncep_gfs_pes10_20200920_20231027.nc"))

In [4]:
# coding=utf-8
# code mostly from https://github.com/jbrowell/HEFTcom24/blob/main/comp_utils.py

"""Rebase-API-Klasse."""

import os
from typing import Literal

from loguru import logger
import pandas as pd
import requests
from requests import Session


class RebaseAPI:
    """Rebase-API-Klasse zum abrufen und abgeben der Daten."""

    challenge_id = 'heftcom2024'
    base_url = 'https://api.rebase.energy'

    def __init__(
        self,
        api_key=None
    ):
        if api_key is None:
            self.api_key = os.environ.get("REBASE_API_KEY")
        else:
            self.api_key = api_key
        self.headers = {
            'Authorization': f"Bearer {self.api_key}"
        }
        self.session = Session()
        self.session.headers = self.headers
    def get_variable(
        self,
        day: str,
        variable: Literal["market_index",
                          "day_ahead_price",
                          "imbalance_price",
                          "wind_total_production",
                          "solar_total_production",
                          "solar_and_wind_forecast"
                          ]
    ) -> pd.DataFrame:
        """
        GET-Request für Challange-Data

        :param day: Datum der abgefragten Daten im Format YYYY-MM-dd
        :param variable: Variable, die abgefragt werden soll
        :return: abgefragte Daten
        :rtype: DataFrame
        """
        url = f"{self.base_url}/challenges/data/{variable}"
        params = {'day': day}
        logger.debug("GET from {}, day={}", url, day)
        resp = self.session.get(url, params=params)
        logger.debug("statuscode={}", resp.status_code)

        data = resp.json()
        df = pd.DataFrame(data)
        return df

    def get_solar_wind_forecast(self, day: str) -> pd.DataFrame:
        """
        GET-Request für Solar/Wind-Vorhersagen

        :param day: Datum der abgefragten Daten
        :return: abgefragte Daten
        :rtype: DataFrame
        """
        url = f"{self.base_url}/challenges/data/solar_and_wind_forecast"
        params = {'day': day}
        logger.debug("GET from {}, day={}", url, day)
        resp = self.session.get(url, params=params)
        logger.debug("statuscode={}", resp.status_code)

        data = resp.json()
        df = pd.DataFrame(data)
        return df

    def get_day_ahead_demand_forecast(self) -> list:
        """
        GET-Request für vorhergesagten Bedarf des nächsten Tages

        :return: abgefragte Daten
        :rtype: Liste mit JSON-encodierten Responses
        """
        url = f"{self.base_url}/challenges/data/day_ahead_demand"
        logger.debug("GET from {}", url)
        resp = self.session.get(url)
        logger.debug("statuscode={}", resp.status_code)

        return resp.json()

    def get_margin_forecast(self) -> list:
        """
        GET-Request für vorhergesagte Marge

        :return: abgefragte Daten
        :rtype: Liste mit JSON-encodierten Responses
        """
        url = f"{self.base_url}/challenges/data/margin_forecast"
        logger.debug("GET from {}", url)
        resp = self.session.get(url)
        logger.debug("statuscode={}", resp.status_code)

        return resp.json()

    def query_weather_latest(
        self,
        model: Literal["DWD_ICON-EU", "NCEP_GFS"],
        lats: list[float],
        lons: list[float],
        variables: str,
        query_type: Literal["grid", "points"]
    ) -> list:
        """
        POST-Request zum abfragen der Wetterdaten

        :param model: genutztes Datenset
        :param lats: Liste der Breitengrade
        :param lons: Liste der Längengrade
        :param variables: Liste der Wettervariablen
        :param query_type: Format, wie Daten zurückgegeben werden
        :return: abgefragte Daten
        :rtype: Liste mit JSON-encodierten Responses
        """
        url = f"{self.base_url}/weather/v2/query"
        body = {
            'model': model,
            'latitude': lats,
            'longitude': lons,
            'variables': variables,
            'type': query_type,
            'output-format': 'json',
            'forecast-horizon': 'latest'
        }
        logger.debug("POST from {}, model={}, lat={}, long={}, "
                     "variables={}, type={}",
                     url, model, lats, lons, variables, query_type)
        resp = requests.post(
            url, json=body, headers={'Authorization': self.api_key}
        )
        logger.debug("statuscode={}", resp.status_code)

        return resp.json()

    def query_weather_latest_points(
        self,
        model: Literal["DWD_ICON-EU", "NCEP_GFS"],
        lats: list[float],
        lons: list[float],
        variables: str,
    ) -> pd.DataFrame:
        """ Daten werden als Liste zurückgegeben"""

        data = self.query_weather_latest(
            model, lats, lons, variables, 'points'
        )

        df = pd.DataFrame()
        for point in range(len(data)):
            new_df = pd.DataFrame(data[point])
            new_df["point"] = point
            new_df["latitude"] = lats[point]
            new_df["longitude"] = lons[point]
            df = pd.concat([df, new_df])

        return df

    def query_weather_latest_grid(
        self,
        model: Literal["DWD_ICON-EU", "NCEP_GFS"],
        lats: list[float],
        lons: list[float],
        variables: str,
    ) -> pd.DataFrame:
        """Daten werden 'eben' zurückgegeben"""

        data = self.query_weather_latest(model, lats, lons, variables, 'grid')
        df = pd.DataFrame(data)

        return df

    def get_hornsea_dwd(self):
        """
        Wetterdaten bei Hornsea1-Koordinaten von DWD_ICON-EU
        als 6x6 Grid
        """

        lats = [53.77, 53.84, 53.9, 53.97, 54.03, 54.1]
        lons = [1.702, 1.767, 1.832, 1.897, 1.962, 2.027]

        variables = "WindSpeed, WindSpeed:100, WindDirection, " \
                    "WindDirection:100, Temperature, RelativeHumidity"
        return self.query_weather_latest_grid(
            'DWD_ICON-EU', lats, lons, variables
        )

    def get_hornsea_gfs(self):
        """
        Wetterdaten bei Hornsea1-Koordinaten von NCEP_GFS
        als 3x3 Grid
        """

        lats = [53.59, 53.84, 54.09]
        lons = [1.522, 1.772, 2.022]

        variables = "WindSpeed, WindSpeed:100, WindDirection, " \
                    "WindDirection:100, Temperature, RelativeHumidity"
        return self.query_weather_latest_grid(
            'NCEP_GFS', lats, lons, variables
        )

    def get_pes10_nwp(self, model: Literal["DWD_ICON-EU", "NCEP_GFS"]):
        """Wetterdaten für Solar"""

        lats = [52.4872562, 52.8776682, 52.1354277, 52.4880497, 51.9563696,
                52.2499177, 52.6416477, 52.2700912, 52.1960768, 52.7082618,
                52.4043468, 52.0679429, 52.024023, 52.7681276, 51.8750506,
                52.5582373, 52.4478922, 52.5214863, 52.8776682, 52.0780721]
        lons = [0.4012455, 0.7906532, -0.2640343, -0.1267052, 0.6588173,
                1.3894081, 1.3509559, 0.7082557, 0.1534462, 0.7302284,
                1.0762977, 1.1751747, 0.2962684, 0.1699257, 0.9115028,
                0.7137489, 0.1204872, 1.5706825, 1.1916542, -0.0113488]

        variables = 'SolarDownwardRadiation, CloudCover, Temperature'
        return self.query_weather_latest_points(model, lats, lons, variables)

    def get_demand_nwp(self, model: Literal["DWD_ICON-EU", "NCEP_GFS"]):
        """Wetterdaten für Bedarf"""

        lats = [51.479, 51.453, 52.449, 53.175, 55.86, 53.875, 54.297]
        lons = [-0.451, -2.6, -1.926, -2.986, -4.264, -0.442, -1.533]

        variables = "Temperature, WindSpeed, WindDirection, " \
                    "TotalPrecipitation, RelativeHumidity"
        return self.query_weather_latest_points(model, lats, lons, variables)

    def submit(self, data) -> None:
        """Ergebnisse zur API schicken."""

        url = f"{self.base_url}/challenges/{self.challenge_id}/submit"

        resp = self.session.post(url, headers=self.headers, json=data)

        print(resp)
        print(resp.text)

        # Write log file
        with open(
            f"logs/sub_{pd.Timestamp('today').strftime('%Y%m%d-%H%M%S')}.txt",
            mode="w",
            encoding="utf-8"
        ) as text_file:
            text_file.write(resp.text)

if __name__=="__main__":
    api = RebaseAPI(api_key='C4A_cdZx0QEUruRLHcy0ZTdcHVbkTJQYvZ-cF96Bl-Y')
    df_hornsea = api.get_hornsea_dwd()
    display(df_hornsea.head(3))
    df_pes10 = api.get_pes10_nwp(model="DWD_ICON-EU")
    display(df_pes10.head(3))
    df_demand = api.get_demand_nwp(model="DWD_ICON-EU")
    display(df_demand.head(3))

2024-10-14 10:38:46.027 | DEBUG    | __main__:query_weather_latest:138 - POST from https://api.rebase.energy/weather/v2/query, model=DWD_ICON-EU, lat=[53.77, 53.84, 53.9, 53.97, 54.03, 54.1], long=[1.702, 1.767, 1.832, 1.897, 1.962, 2.027], variables=WindSpeed, WindSpeed:100, WindDirection, WindDirection:100, Temperature, RelativeHumidity, type=grid
2024-10-14 10:38:46.852 | DEBUG    | __main__:query_weather_latest:144 - statuscode=200


,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,ref_datetime,valid_datetime
0,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,66.316353,65.688858,...,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918,"Mon, 14 Oct 2024 00:00:00 GMT","Mon, 14 Oct 2024 00:00:00 GMT"
1,67.022293,66.786980,66.865417,66.786980,66.865417,66.708542,66.708542,66.708542,66.865417,66.786980,...,1.911245,1.948990,1.882661,1.974484,2.079465,2.168201,2.231393,2.238368,"Mon, 14 Oct 2024 00:00:00 GMT","Mon, 14 Oct 2024 01:00:00 GMT"
2,64.590729,65.061356,65.610420,65.767288,66.237915,66.316353,65.845726,65.924164,66.551666,66.630104,...,1.129630,1.243597,1.768376,1.746835,1.717932,1.744587,1.815539,1.905149,"Mon, 14 Oct 2024 00:00:00 GMT","Mon, 14 Oct 2024 02:00:00 GMT"


2024-10-14 10:38:46.896 | DEBUG    | __main__:query_weather_latest:138 - POST from https://api.rebase.energy/weather/v2/query, model=DWD_ICON-EU, lat=[52.4872562, 52.8776682, 52.1354277, 52.4880497, 51.9563696, 52.2499177, 52.6416477, 52.2700912, 52.1960768, 52.7082618, 52.4043468, 52.0679429, 52.024023, 52.7681276, 51.8750506, 52.5582373, 52.4478922, 52.5214863, 52.8776682, 52.0780721], long=[0.4012455, 0.7906532, -0.2640343, -0.1267052, 0.6588173, 1.3894081, 1.3509559, 0.7082557, 0.1534462, 0.7302284, 1.0762977, 1.1751747, 0.2962684, 0.1699257, 0.9115028, 0.7137489, 0.1204872, 1.5706825, 1.1916542, -0.0113488], variables=SolarDownwardRadiation, CloudCover, Temperature, type=points
2024-10-14 10:38:47.186 | DEBUG    | __main__:query_weather_latest:144 - statuscode=200


,valid_datetime,ref_datetime,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-14T00:00:00Z,2024-10-14T00:00:00Z,0.0,1.0,6.03,0,52.487256,0.401245
1,2024-10-14T01:00:00Z,2024-10-14T00:00:00Z,0.0,1.0,6.14,0,52.487256,0.401245
2,2024-10-14T02:00:00Z,2024-10-14T00:00:00Z,0.0,1.0,6.24,0,52.487256,0.401245


2024-10-14 10:38:47.215 | DEBUG    | __main__:query_weather_latest:138 - POST from https://api.rebase.energy/weather/v2/query, model=DWD_ICON-EU, lat=[51.479, 51.453, 52.449, 53.175, 55.86, 53.875, 54.297], long=[-0.451, -2.6, -1.926, -2.986, -4.264, -0.442, -1.533], variables=Temperature, WindSpeed, WindDirection, TotalPrecipitation, RelativeHumidity, type=points
2024-10-14 10:38:47.844 | DEBUG    | __main__:query_weather_latest:144 - statuscode=200


,valid_datetime,ref_datetime,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity,point,latitude,longitude
0,2024-10-14T00:00:00Z,2024-10-14T00:00:00Z,7.54,2.23,76.54,NaN,84.12,0,51.479,-0.451
1,2024-10-14T01:00:00Z,2024-10-14T00:00:00Z,7.64,2.56,83.11,0.41,82.24,0,51.479,-0.451
2,2024-10-14T02:00:00Z,2024-10-14T00:00:00Z,7.64,2.64,77.29,0.77,84.91,0,51.479,-0.451


In [5]:
same_columns_df1_df2 = df_dwd_demand.columns.equals(df_demand.columns)
same_columns_df3_df4 = df_dwd_hornsea.columns.equals(df_hornsea.columns)
same_columns_df5_df6 = df_dwd_pes10.columns.equals(df_pes10.columns)

# Überprüfen auf gleiche Spalten und Unterschiede
def compare_columns(df1, df2):
    same_columns = df1.columns.equals(df2.columns)
    diff_columns_df1 = set(df1.columns) - set(df2.columns)
    diff_columns_df2 = set(df2.columns) - set(df1.columns)
    return same_columns, diff_columns_df1, diff_columns_df2

# Vergleich durchführen
same_columns_df1_df2, diff_df1_df2_1, diff_df1_df2_2 = compare_columns(df_dwd_demand, df_demand)
same_columns_df3_df4, diff_df3_df4_1, diff_df3_df4_2 = compare_columns(df_dwd_hornsea, df_hornsea)
same_columns_df5_df6, diff_df5_df6_1, diff_df5_df6_2 = compare_columns(df_dwd_pes10, df_pes10)

# Ergebnisse ausgeben
print(f"df_dwd_demand und df_demand haben die gleichen Spalten: {same_columns_df1_df2}")
print(f"Unterschiedliche Spalten in df_dwd_demand: {diff_df1_df2_1}")
print(f"Unterschiedliche Spalten in df_demand: {diff_df1_df2_2}\n")

print(f"df_dwd_hornsea und df_hornsea haben die gleichen Spalten: {same_columns_df3_df4}")
print(f"Unterschiedliche Spalten in df_dwd_hornsea: {diff_df3_df4_1}")
print(f"Unterschiedliche Spalten in df_hornsea: {diff_df3_df4_2}\n")

print(f"df_dwd_pes10 und df_pes10 haben die gleichen Spalten: {same_columns_df5_df6}")
print(f"Unterschiedliche Spalten in df_dwd_pes10: {diff_df5_df6_1}")
print(f"Unterschiedliche Spalten in df_pes10: {diff_df5_df6_2}")

df_dwd_demand und df_demand haben die gleichen Spalten: False
Unterschiedliche Spalten in df_dwd_demand: set()
Unterschiedliche Spalten in df_demand: set()

df_dwd_hornsea und df_hornsea haben die gleichen Spalten: False
Unterschiedliche Spalten in df_dwd_hornsea: {'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100', 'RelativeHumidity', 'Temperature', 'latitude', 'longitude'}
Unterschiedliche Spalten in df_hornsea: {'Temperature_latitude_53.84_longitude_2.027', 'Temperature_latitude_54.1_longitude_1.962', 'WindDirection_latitude_53.84_longitude_1.767', 'WindDirection:100_latitude_53.84_longitude_1.702', 'WindDirection:100_latitude_53.77_longitude_1.962', 'RelativeHumidity_latitude_53.77_longitude_1.767', 'WindDirection_latitude_54.03_longitude_1.702', 'WindDirection:100_latitude_53.9_longitude_1.767', 'WindDirection:100_latitude_53.97_longitude_1.897', 'WindDirection_latitude_53.9_longitude_1.962', 'Temperature_latitude_53.84_longitude_1.832', 'WindDirection:100_latitude

In [6]:
# Rename der Spaltennamen
rename = {'valid_datetime' : 'valid_time','ref_datetime' : 'reference_time'}
df_hornsea = df_hornsea.rename(columns=rename)
display(df_hornsea.head(2))
df_pes10 = df_pes10.rename(columns=rename)
display(df_pes10.head(2))
df_demand = df_demand.rename(columns=rename)
display(df_demand.head(2))

,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,reference_time,valid_time
0,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,66.316353,65.688858,...,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918,"Mon, 14 Oct 2024 00:00:00 GMT","Mon, 14 Oct 2024 00:00:00 GMT"
1,67.022293,66.786980,66.865417,66.786980,66.865417,66.708542,66.708542,66.708542,66.865417,66.786980,...,1.911245,1.948990,1.882661,1.974484,2.079465,2.168201,2.231393,2.238368,"Mon, 14 Oct 2024 00:00:00 GMT","Mon, 14 Oct 2024 01:00:00 GMT"


,valid_time,reference_time,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-14T00:00:00Z,2024-10-14T00:00:00Z,0.0,1.0,6.03,0,52.487256,0.401245
1,2024-10-14T01:00:00Z,2024-10-14T00:00:00Z,0.0,1.0,6.14,0,52.487256,0.401245


,valid_time,reference_time,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity,point,latitude,longitude
0,2024-10-14T00:00:00Z,2024-10-14T00:00:00Z,7.54,2.23,76.54,NaN,84.12,0,51.479,-0.451
1,2024-10-14T01:00:00Z,2024-10-14T00:00:00Z,7.64,2.56,83.11,0.41,82.24,0,51.479,-0.451


In [7]:
df_hornsea.head(2)

,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,reference_time,valid_time
0,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,66.316353,65.688858,...,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918,"Mon, 14 Oct 2024 00:00:00 GMT","Mon, 14 Oct 2024 00:00:00 GMT"
1,67.022293,66.786980,66.865417,66.786980,66.865417,66.708542,66.708542,66.708542,66.865417,66.786980,...,1.911245,1.948990,1.882661,1.974484,2.079465,2.168201,2.231393,2.238368,"Mon, 14 Oct 2024 00:00:00 GMT","Mon, 14 Oct 2024 01:00:00 GMT"


In [8]:
# Liste der DataFrames und Spalten, die umgewandelt werden sollen
dfs = [df_hornsea]# df_pes10, df_demand]
spalten_liste = ['valid_time', 'reference_time']

# Umwandlung der Spalten in datetime64
for df in dfs:
    for spalte in spalten_liste:
        if spalte in df.columns: 
            df[spalte] = pd.to_datetime(df[spalte], format="%a, %d %b %Y %H:%M:%S %Z", utc=False)

# Ergebnis überprüfen
for i, df in enumerate(dfs, start=1):
    print(f"df{i} nach der Umwandlung:")
    display(df.head())
    display(df.dtypes)
    print("\n")


df1 nach der Umwandlung:


,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,reference_time,valid_time
0,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,66.316353,65.688858,...,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00
1,67.022293,66.786980,66.865417,66.786980,66.865417,66.708542,66.708542,66.708542,66.865417,66.786980,...,1.911245,1.948990,1.882661,1.974484,2.079465,2.168201,2.231393,2.238368,2024-10-14 00:00:00+00:00,2024-10-14 01:00:00+00:00
2,64.590729,65.061356,65.610420,65.767288,66.237915,66.316353,65.845726,65.924164,66.551666,66.630104,...,1.129630,1.243597,1.768376,1.746835,1.717932,1.744587,1.815539,1.905149,2024-10-14 00:00:00+00:00,2024-10-14 02:00:00+00:00
3,62.472923,62.472923,63.100422,64.041672,64.669167,65.139793,63.649483,64.433853,65.767288,65.531982,...,1.079481,1.129495,1.589406,1.578860,1.678738,1.762863,1.893993,1.987890,2024-10-14 00:00:00+00:00,2024-10-14 03:00:00+00:00
4,61.061050,61.688553,61.531677,61.374802,61.845425,62.159176,61.845425,62.472923,62.472923,62.865112,...,2.088983,2.035767,1.876758,1.625820,1.384023,1.348718,1.385250,1.419942,2024-10-14 00:00:00+00:00,2024-10-14 04:00:00+00:00


RelativeHumidity_latitude_53.77_longitude_1.702                float64
RelativeHumidity_latitude_53.77_longitude_1.767                float64
RelativeHumidity_latitude_53.77_longitude_1.832                float64
RelativeHumidity_latitude_53.77_longitude_1.897                float64
RelativeHumidity_latitude_53.77_longitude_1.962                float64
                                                          ...         
WindSpeed_latitude_54.1_longitude_1.897                        float64
WindSpeed_latitude_54.1_longitude_1.962                        float64
WindSpeed_latitude_54.1_longitude_2.027                        float64
reference_time                                     datetime64[ns, UTC]
valid_time                                         datetime64[ns, UTC]
Length: 218, dtype: object

In [9]:
# Liste der DataFrames und Spalten, die umgewandelt werden sollen
dfs = [df_pes10, df_demand]
spalten_liste = ['valid_time', 'reference_time']

# Umwandlung der Spalten in datetime64
for df in dfs:
    for spalte in spalten_liste:
        if spalte in df.columns: 
            df[spalte] = pd.to_datetime(df[spalte])

# Ergebnis überprüfen
for i, df in enumerate(dfs, start=1):
    print(f"df{i} nach der Umwandlung:")
    display(df.head())
    display(df.dtypes)
    print("\n")

df1 nach der Umwandlung:


,valid_time,reference_time,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.03,0,52.487256,0.401245
1,2024-10-14 01:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.14,0,52.487256,0.401245
2,2024-10-14 02:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.24,0,52.487256,0.401245
3,2024-10-14 03:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.03,0,52.487256,0.401245
4,2024-10-14 04:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.03,0,52.487256,0.401245


valid_time                datetime64[ns, UTC]
reference_time            datetime64[ns, UTC]
SolarDownwardRadiation                float64
CloudCover                            float64
Temperature                           float64
point                                   int64
latitude                              float64
longitude                             float64
dtype: object



df2 nach der Umwandlung:


,valid_time,reference_time,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity,point,latitude,longitude
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,7.54,2.23,76.54,NaN,84.12,0,51.479,-0.451
1,2024-10-14 01:00:00+00:00,2024-10-14 00:00:00+00:00,7.64,2.56,83.11,0.41,82.24,0,51.479,-0.451
2,2024-10-14 02:00:00+00:00,2024-10-14 00:00:00+00:00,7.64,2.64,77.29,0.77,84.91,0,51.479,-0.451
3,2024-10-14 03:00:00+00:00,2024-10-14 00:00:00+00:00,7.75,2.72,73.41,0.89,86.95,0,51.479,-0.451
4,2024-10-14 04:00:00+00:00,2024-10-14 00:00:00+00:00,7.97,2.79,77.62,2.97,88.04,0,51.479,-0.451


valid_time            datetime64[ns, UTC]
reference_time        datetime64[ns, UTC]
Temperature                       float64
WindSpeed                         float64
WindDirection                     float64
TotalPrecipitation                float64
RelativeHumidity                  float64
point                               int64
latitude                          float64
longitude                         float64
dtype: object

In [10]:
display(df_hornsea.head(2))
display(df_pes10.head(2))
display(df_demand.head(2))

,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,reference_time,valid_time
0,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,66.316353,65.688858,...,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00
1,67.022293,66.786980,66.865417,66.786980,66.865417,66.708542,66.708542,66.708542,66.865417,66.786980,...,1.911245,1.948990,1.882661,1.974484,2.079465,2.168201,2.231393,2.238368,2024-10-14 00:00:00+00:00,2024-10-14 01:00:00+00:00


,valid_time,reference_time,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.03,0,52.487256,0.401245
1,2024-10-14 01:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.14,0,52.487256,0.401245


,valid_time,reference_time,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity,point,latitude,longitude
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,7.54,2.23,76.54,NaN,84.12,0,51.479,-0.451
1,2024-10-14 01:00:00+00:00,2024-10-14 00:00:00+00:00,7.64,2.56,83.11,0.41,82.24,0,51.479,-0.451


## Datentypen

In [11]:
print(f"df_hornsea\n{df_hornsea.dtypes}")

df_hornsea
RelativeHumidity_latitude_53.77_longitude_1.702                float64
RelativeHumidity_latitude_53.77_longitude_1.767                float64
RelativeHumidity_latitude_53.77_longitude_1.832                float64
RelativeHumidity_latitude_53.77_longitude_1.897                float64
RelativeHumidity_latitude_53.77_longitude_1.962                float64
                                                          ...         
WindSpeed_latitude_54.1_longitude_1.897                        float64
WindSpeed_latitude_54.1_longitude_1.962                        float64
WindSpeed_latitude_54.1_longitude_2.027                        float64
reference_time                                     datetime64[ns, UTC]
valid_time                                         datetime64[ns, UTC]
Length: 218, dtype: object


In [12]:
print(f"df_pes10\n{df_pes10.dtypes}")


df_pes10
valid_time                datetime64[ns, UTC]
reference_time            datetime64[ns, UTC]
SolarDownwardRadiation                float64
CloudCover                            float64
Temperature                           float64
point                                   int64
latitude                              float64
longitude                             float64
dtype: object


In [13]:
print(f"df_demand\n{df_demand.dtypes}")

df_demand
valid_time            datetime64[ns, UTC]
reference_time        datetime64[ns, UTC]
Temperature                       float64
WindSpeed                         float64
WindDirection                     float64
TotalPrecipitation                float64
RelativeHumidity                  float64
point                               int64
latitude                          float64
longitude                         float64
dtype: object


# Vorverarbeitung

In [14]:
df_hornsea.head()

,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,reference_time,valid_time
0,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,66.316353,65.688858,...,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00
1,67.022293,66.786980,66.865417,66.786980,66.865417,66.708542,66.708542,66.708542,66.865417,66.786980,...,1.911245,1.948990,1.882661,1.974484,2.079465,2.168201,2.231393,2.238368,2024-10-14 00:00:00+00:00,2024-10-14 01:00:00+00:00
2,64.590729,65.061356,65.610420,65.767288,66.237915,66.316353,65.845726,65.924164,66.551666,66.630104,...,1.129630,1.243597,1.768376,1.746835,1.717932,1.744587,1.815539,1.905149,2024-10-14 00:00:00+00:00,2024-10-14 02:00:00+00:00
3,62.472923,62.472923,63.100422,64.041672,64.669167,65.139793,63.649483,64.433853,65.767288,65.531982,...,1.079481,1.129495,1.589406,1.578860,1.678738,1.762863,1.893993,1.987890,2024-10-14 00:00:00+00:00,2024-10-14 03:00:00+00:00
4,61.061050,61.688553,61.531677,61.374802,61.845425,62.159176,61.845425,62.472923,62.472923,62.865112,...,2.088983,2.035767,1.876758,1.625820,1.384023,1.348718,1.385250,1.419942,2024-10-14 00:00:00+00:00,2024-10-14 04:00:00+00:00


In [15]:
df_hornsea.head()

,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,RelativeHumidity_latitude_53.84_longitude_1.832,RelativeHumidity_latitude_53.84_longitude_1.897,...,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027,reference_time,valid_time
0,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,66.316353,65.688858,...,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00
1,67.022293,66.786980,66.865417,66.786980,66.865417,66.708542,66.708542,66.708542,66.865417,66.786980,...,1.911245,1.948990,1.882661,1.974484,2.079465,2.168201,2.231393,2.238368,2024-10-14 00:00:00+00:00,2024-10-14 01:00:00+00:00
2,64.590729,65.061356,65.610420,65.767288,66.237915,66.316353,65.845726,65.924164,66.551666,66.630104,...,1.129630,1.243597,1.768376,1.746835,1.717932,1.744587,1.815539,1.905149,2024-10-14 00:00:00+00:00,2024-10-14 02:00:00+00:00
3,62.472923,62.472923,63.100422,64.041672,64.669167,65.139793,63.649483,64.433853,65.767288,65.531982,...,1.079481,1.129495,1.589406,1.578860,1.678738,1.762863,1.893993,1.987890,2024-10-14 00:00:00+00:00,2024-10-14 03:00:00+00:00
4,61.061050,61.688553,61.531677,61.374802,61.845425,62.159176,61.845425,62.472923,62.472923,62.865112,...,2.088983,2.035767,1.876758,1.625820,1.384023,1.348718,1.385250,1.419942,2024-10-14 00:00:00+00:00,2024-10-14 04:00:00+00:00


In [16]:
# # 491319 rows
# df_hornsea_test = df_hornsea
# df_hornsea_test = df_hornsea_test.groupby(["reference_time", "valid_time"]).mean().reset_index()#.drop(columns=["point", "longitude", "latitude"])
# # 1273203 rows
# df_hornsea = df_hornsea.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()
# df_hornsea

In [17]:
dataframes = [df_pes10, df_demand, df_hornsea]  # Liste der DataFrames

# Gruppieren und Mittelwert berechnen für beide DataFrames
dataframes = [df.groupby(["reference_time", "valid_time"]).mean().reset_index() for df in dataframes]

# Interpolieren und bestimmte Spalten entfernen für beide DataFrames
dataframes = [
    df.set_index("valid_time")
      .groupby(["reference_time"])
      .resample("30min")
      .interpolate("linear")
      .drop(columns=[col for col in ["reference_time", "point", "longitude", "latitude"] if col in df.columns])  # Droppen der gewünschten Spalten, wenn vorhanden
      .reset_index() 
    for df in dataframes
]

# Zuweisung zurück zu den originalen Variablen
df_pes10, df_demand, df_hornsea = dataframes


In [18]:
df_pes10.head(2)

,reference_time,valid_time,SolarDownwardRadiation,CloudCover,Temperature
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.379
1,2024-10-14 00:00:00+00:00,2024-10-14 00:30:00+00:00,0.0,1.0,6.395


In [19]:
df_demand.head(2)

,reference_time,valid_time,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,5.968571,1.910000,117.467143,NaN,88.468571
1,2024-10-14 00:00:00+00:00,2024-10-14 00:30:00+00:00,5.936429,1.857857,112.609286,NaN,88.468571


In [20]:
df_hornsea.head(2)

,reference_time,valid_time,RelativeHumidity_latitude_53.77_longitude_1.702,RelativeHumidity_latitude_53.77_longitude_1.767,RelativeHumidity_latitude_53.77_longitude_1.832,RelativeHumidity_latitude_53.77_longitude_1.897,RelativeHumidity_latitude_53.77_longitude_1.962,RelativeHumidity_latitude_53.77_longitude_2.027,RelativeHumidity_latitude_53.84_longitude_1.702,RelativeHumidity_latitude_53.84_longitude_1.767,...,WindSpeed_latitude_54.03_longitude_1.832,WindSpeed_latitude_54.03_longitude_1.897,WindSpeed_latitude_54.03_longitude_1.962,WindSpeed_latitude_54.03_longitude_2.027,WindSpeed_latitude_54.1_longitude_1.702,WindSpeed_latitude_54.1_longitude_1.767,WindSpeed_latitude_54.1_longitude_1.832,WindSpeed_latitude_54.1_longitude_1.897,WindSpeed_latitude_54.1_longitude_1.962,WindSpeed_latitude_54.1_longitude_2.027
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,66.473228,66.394791,66.551666,66.473228,66.159477,65.845726,66.473228,66.316353,...,1.595246,1.681919,1.740687,1.815981,1.965242,2.021636,2.065150,2.065104,2.097660,2.137918
1,2024-10-14 00:00:00+00:00,2024-10-14 00:30:00+00:00,66.747761,66.590885,66.708542,66.630104,66.512447,66.277134,66.590885,66.512447,...,1.629526,1.743810,1.825966,1.882485,1.923952,1.998060,2.072308,2.116652,2.164526,2.188143


In [21]:
# Dictionary zur Speicherung der einzelnen DataFrames
dataframes = {}

# Durch jede Spalte im ursprünglichen DataFrame iterieren, außer der Zeitspalte
for column in df_hornsea.columns[1:]:  # Zeitspalte überspringen
    # Spaltennamen aufteilen und Anzahl der Teile überprüfen
    parts = column.split('_')
    
    if len(parts) == 5:  # Erwartet 5 Teile: [Messungstyp, "latitude", Latitude, "longitude", Longitude]
        data_type = parts[0]  # "RelativeHumidity" oder "Temperature"
        latitude_value = parts[2]  # Latitude
        longitude_value = parts[4]  # Longitude
        
        # Neuer DataFrame für jede Messung
        temp_df = pd.DataFrame({
            'valid_time': df_hornsea['valid_time'],  # Zeitdaten hinzufügen
            'reference_time': df_hornsea['reference_time'],  # Reference Time hinzufügen
            f'{data_type}': df_hornsea[column],  # Neuer Spaltenname
            'Latitude': latitude_value,
            'Longitude': longitude_value
        })
        
        # Den neuen DataFrame im Dictionary speichern
        df_name = f"{data_type}_{latitude_value}_{longitude_value}"  # Name für den DataFrame
        dataframes[df_name] = temp_df

# Alle DataFrames zu einem großen DataFrame zusammenfassen
combined_df = pd.concat(dataframes.values(), axis=0, ignore_index=True)
display(combined_df)

# Kombinieren der Daten, wo Timestamp, Latitude_Value und Longitude_Value gleich sind
# Alle numerischen Spalten aggregieren, die nicht die Gruppierungsspalten sind
df_hornsea = combined_df.groupby(['valid_time', 'reference_time', 'Latitude', 'Longitude']).agg(
    {col: 'mean' for col in combined_df.columns if col not in ['valid_time', 'reference_time', 'Latitude', 'Longitude']}
).reset_index()

,valid_time,reference_time,RelativeHumidity,Latitude,Longitude,Temperature,WindDirection:100,WindDirection,WindSpeed:100,WindSpeed
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,66.473228,53.77,1.702,NaN,NaN,NaN,NaN,NaN
1,2024-10-14 00:30:00+00:00,2024-10-14 00:00:00+00:00,66.747761,53.77,1.702,NaN,NaN,NaN,NaN,NaN
2,2024-10-14 01:00:00+00:00,2024-10-14 00:00:00+00:00,67.022293,53.77,1.702,NaN,NaN,NaN,NaN,NaN
3,2024-10-14 01:30:00+00:00,2024-10-14 00:00:00+00:00,65.806511,53.77,1.702,NaN,NaN,NaN,NaN,NaN
4,2024-10-14 02:00:00+00:00,2024-10-14 00:00:00+00:00,64.590729,53.77,1.702,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
52051,2024-10-18 22:00:00+00:00,2024-10-14 00:00:00+00:00,NaN,54.1,2.027,NaN,NaN,NaN,NaN,5.968337
52052,2024-10-18 22:30:00+00:00,2024-10-14 00:00:00+00:00,NaN,54.1,2.027,NaN,NaN,NaN,NaN,6.237383
52053,2024-10-18 23:00:00+00:00,2024-10-14 00:00:00+00:00,NaN,54.1,2.027,NaN,NaN,NaN,NaN,6.506428
52054,2024-10-18 23:30:00+00:00,2024-10-14 00:00:00+00:00,NaN,54.1,2.027,NaN,NaN,NaN,NaN,6.775474


##### Überprüfung der Transormation

In [22]:
# Koordinaten speichern
latitude = 54.1
longitude = 2.027

# Liste der Spaltennamen, die du filtern möchtest
columns_to_filter = [
    'reference_time',
    'valid_time',
    f'RelativeHumidity_latitude_{latitude}_longitude_{longitude}',
    f'Temperature_latitude_{latitude}_longitude_{longitude}',
    f'WindDirection:100_latitude_{latitude}_longitude_{longitude}',
    f'WindDirection_latitude_{latitude}_longitude_{longitude}',
    f'WindSpeed:100_latitude_{latitude}_longitude_{longitude}',
    f'WindSpeed_latitude_{latitude}_longitude_{longitude}'
]

# Filtern des DataFrames mit der Liste der Spalten
df_hornsea = df_hornsea.filter(columns_to_filter).tail(2)

# Anzeigen des gefilterten DataFrames
display(df_hornsea)


,reference_time,valid_time
8674,2024-10-14 00:00:00+00:00,2024-10-19 00:00:00+00:00
8675,2024-10-14 00:00:00+00:00,2024-10-19 00:00:00+00:00


In [23]:
df_hornsea.tail(2)

,reference_time,valid_time
8674,2024-10-14 00:00:00+00:00,2024-10-19 00:00:00+00:00
8675,2024-10-14 00:00:00+00:00,2024-10-19 00:00:00+00:00


##### Finale df

In [24]:
display(df_pes10.head(2))
display(df_demand.head(2))
display(df_hornsea.head(2))

,reference_time,valid_time,SolarDownwardRadiation,CloudCover,Temperature
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,0.0,1.0,6.379
1,2024-10-14 00:00:00+00:00,2024-10-14 00:30:00+00:00,0.0,1.0,6.395


,reference_time,valid_time,Temperature,WindSpeed,WindDirection,TotalPrecipitation,RelativeHumidity
0,2024-10-14 00:00:00+00:00,2024-10-14 00:00:00+00:00,5.968571,1.910000,117.467143,NaN,88.468571
1,2024-10-14 00:00:00+00:00,2024-10-14 00:30:00+00:00,5.936429,1.857857,112.609286,NaN,88.468571


,reference_time,valid_time
8674,2024-10-14 00:00:00+00:00,2024-10-19 00:00:00+00:00
8675,2024-10-14 00:00:00+00:00,2024-10-19 00:00:00+00:00
